In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

# Loading text file 
dataset = load_dataset("text", data_files="/Users/tamasmakos/dev/langflow/text_/pdfs/Output/pdf_text.txt")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [8]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("SZTAKI-HLT/hubert-base-cc", num_labels=5)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [10]:
from transformers import Trainer

trainer = Trainer(
    model, 
    training_args, 
    train_dataset=tokenized_datasets["train"], 
    eval_dataset=tokenized_datasets["validation"], 
    data_collator=data_collator, 
    tokenizer=tokenizer)
                                                                                                 
trainer.train()

KeyError: 'validation'

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
                              
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np 
from datasets import load_metric

metric = load_metric("glue", "mrpc")
preds = np.argmax(predictions.predictions, axis=1)
metric. compute(predictions-preds, references=predictions. label_ids)